In [7]:
!pip install langchain
!pip install langchain-openai

In [8]:
import yaml
with open('chatgpt-api-credentials.yml') as file:
    api_creds = yaml.safe_load(file)

In [9]:
api_creds.keys()

dict_keys(['OPENAI-API-KEY'])

In [10]:
import os
os.environ['OPENAI_API_KEY'] = api_creds['OPENAI-API-KEY']

In [11]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model_name='gpt-4o', temperature=0.0)

In [13]:
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from operator import itemgetter

def run_chatgpt_prompt(system_prompt='',history_window=30,temperature=0.3,llm=model):
    if system_prompt:
        SYS_PROMPT = system_prompt
    else:
        SYS_PROMPT = """
        Act as a helpful AI Assistant
        """
    prompt = ChatPromptTemplate.from_messages(
        [
            ('system',SYS_PROMPT),
            MessagesPlaceholder(variable_name="history"),
            ('human','{input}')
        ]
    )
    memory = ConversationBufferWindowMemory(k=history_window,return_messages=True)
    conversation_chain = (
        RunnablePassthrough.assign(
            history=RunnableLambda(memory.load_memory_variables)
            |
            itemgetter("history")
        )
        |
        prompt
        |
        llm
    )
    print("Hello! I'm your friendly chatbot. Let's chat (Type 'STOP' to end)")
    while True:
        prompt = input("USER: >>>")
        if prompt.strip().upper() == 'STOP':
            print("ChatGPT: >>> Goodbye!")
            break
        user_input = {'input':prompt}
        reply = conversation_chain.invoke(user_input)
        print(f"ChatGPT: >>>\n {reply.content}")
        memory.save_context(user_input,{'output':reply.content})
        

In [ ]:
run_chatgpt_prompt()

Hello! I'm your friendly chatbot. Let's chat (Type 'STOP' to end)


USER: >>> what is 10*3?


ChatGPT: >>>
 10 multiplied by 3 equals 30.


USER: >>> tell me a joke about india


ChatGPT: >>>
 Sure, here's a light-hearted joke about India:

Why did the scarecrow become a successful Bollywood actor?

Because he was outstanding in his field!
